## Responses API란 무엇인가요?

Responses API는 OpenAI 모델과 상호작용하는 새로운 방식으로, 이전 API보다 더 간단하고 유연하게 설계되었습니다. 이 API는 여러 도구를 사용하고, 다중 턴 대화를 처리하며, 텍스트뿐만 아니라 다양한 유형의 데이터를 다룰 수 있는 고급 AI 애플리케이션을 쉽게 구축할 수 있도록 합니다.

Chat Completions와 같이 주로 텍스트를 위해 설계된 이전 API나 설정이 많이 필요한 Assistants API와 달리, Responses API는 다음을 위해 처음부터 설계되었습니다:

- 원활한 다중 턴 상호작용 (단일 API 호출에서 여러 단계의 대화를 이어갈 수 있음)
- 강력한 호스팅 도구에 대한 쉬운 접근 (파일 검색, 웹 검색, 코드 해석기 등)
- 모델에 보낼 컨텍스트에 대한 세밀한 제어

AI 모델이 점점 더 복잡하고 장기적인 추론을 수행할 수 있게 되면서, 비동기적이고 상태를 유지할 수 있는 API가 필요합니다. Responses API는 이러한 요구를 충족하도록 설계되었습니다.

이 가이드에서는 Responses API가 제공하는 새로운 기능과 함께 시작하는 데 도움이 되는 실용적인 예제를 살펴봅니다.

## 기본 사항

Responses API는 표면적으로 Completions API와 매우 유사하게 설계되었습니다.

In [4]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2025-03-01-preview"
)

In [5]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="농담 하나 해줘",
)

In [6]:
print(response.output[0].content[0].text)

왜 컴퓨터는 바다를 싫어할까요?

왜냐하면, 많은 버그가 있기 때문이죠! 😂


Responses API의 주요 기능 중 하나는 상태를 유지한다는 점입니다. 즉, 대화의 상태를 직접 관리할 필요가 없으며, API가 이를 대신 처리합니다. 예를 들어, 언제든지 응답을 가져오면 전체 대화 기록이 포함됩니다.

In [7]:
fetched_response = client.responses.retrieve(
    response_id=response.id
)

print(fetched_response.output[0].content[0].text)

왜 컴퓨터는 바다를 싫어할까요?

왜냐하면, 많은 버그가 있기 때문이죠! 😂


이전 응답을 참조하여 대화를 계속할 수 있습니다.

In [8]:
response_two = client.responses.create(
    model="gpt-4o-mini",
    input="다른 농담 하나 더 해줘",
    previous_response_id=response.id
)

In [9]:
print(response_two.output[0].content[0].text)

왜 고양이는 자주 컴퓨터 위에 앉아 있을까요?

마우스가 잡고 싶어서요! 🐱🖱️😄


물론 컨텍스트를 직접 관리할 수도 있습니다. 하지만 OpenAI가 컨텍스트를 유지해 주는 이점 중 하나는 응답을 원하는 시점에서 분기(fork)하여 해당 지점에서 대화를 계속할 수 있다는 점입니다.

In [10]:
response_two_forked = client.responses.create(
    model="gpt-4o-mini",
    input="그 농담은 별로였어. 다른 농담 하나 더 해주고 두 농담의 차이점도 알려줘",
    previous_response_id=response.id  # 첫 번째 응답에서 분기하여 계속 진행
)

output_text = response_two_forked.output[0].content[0].text
print(output_text)

알겠어요! 이번에는 이런 농담은 어때요?

왜 고양이는 컴퓨터를 잘 사용할까요?

왜냐하면 마우스를 잘 다루거든요! 😺

### 두 농담의 차이점:
1. **주제**: 첫 번째 농담은 컴퓨터와 바다의 비유적인 관계에 초점을 맞추었고, 두 번째 농담은 고양이가 컴퓨터와 마우스와 관련된 유머를 사용했습니다.
2. **유머의 형태**: 첫 번째 농담은 언어 유희(버그)였고, 두 번째 농담은 동물의 특성과 관련된 농담으로, 고양이의 행동을 활용했습니다.

어떤 게 더 마음에 드세요?


## 호스팅 도구

Responses API의 또 다른 이점은 `file_search` 및 `web_search`와 같은 호스팅 도구를 지원한다는 점입니다. 도구를 수동으로 호출하는 대신, 도구를 전달하면 API가 어떤 도구를 사용할지 결정하고 이를 사용합니다.

다음은 `web_search` 도구를 사용하여 웹 검색 결과를 응답에 통합하는 예제입니다.

In [17]:
client = AzureOpenAI(
  azure_endpoint="https://admin-ma37dffz-japaneast.openai.azure.com", 
  api_key="AhTQPqAuZjVZjepC8v5L683LABhYfULzQax8FKVSklBzIguaIvlfJQQJ99BDACi0881XJ3w3AAAAACOGNzCD",  
  api_version="2025-03-01-preview",
 
)

response = client.responses.create(
    model="gpt-4o",  # 또는 다른 지원 모델
    input="AI에 대한 최신 뉴스는 무엇인가요?",
    tools=[
        {
            "type": "web_search"
        }
    ]
)

BadRequestError: Error code: 400 - {'error': {'message': "Hosted tool 'web_search_preview' is not supported with gpt-4o-2024-11-20.", 'type': 'invalid_request_error', 'param': 'tools', 'code': None}}

In [17]:
import json
print(json.dumps(response.output, default=lambda o: o.__dict__, indent=2))

[
  {
    "id": "ws_67bd64fe91f081919bec069ad65797f1",
    "status": "completed",
    "type": "web_search_call"
  },
  {
    "id": "msg_67bd6502568c8191a2cbb154fa3fbf4c",
    "content": [
      {
        "annotations": [
          {
            "index": null,
            "title": "Huawei improves AI chip production in boost for China's tech goals",
            "type": "url_citation",
            "url": "https://www.ft.com/content/f46b7f6d-62ed-4b64-8ad7-2417e5ab34f6?utm_source=chatgpt.com"
          },
          {
            "index": null,
            "title": "Apple cheers Trump with $500bn US investment plan; more losses on Wall Street - as it happened",
            "type": "url_citation",
            "url": "https://www.theguardian.com/business/live/2025/feb/24/euro-hits-one-month-high-german-election-result-stock-markets-dax-bank-of-england-business-live-news?utm_source=chatgpt.com"
          },
          {
            "index": null,
            "title": "Microsoft axes data cente

## 멀티모달, 도구 확장 대화

Responses API는 텍스트, 이미지, 오디오 모달리티를 기본적으로 지원합니다.  
모든 것을 결합하여 Responses API를 통해 단일 API 호출로 완전한 멀티모달, 도구 확장 상호작용을 구축할 수 있습니다.

In [18]:
import base64

from IPython.display import Image, display

# 제공된 URL에서 이미지를 표시
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"
display(Image(url=url, width=400))

response_multimodal = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": 
                 "이미지와 관련된 키워드를 생성하고, 검색 도구를 사용하여 키워드와 관련된 뉴스를 웹에서 검색하세요. "
                 "결과를 요약하고 출처를 인용하세요."},
                {"type": "input_image", "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"}
            ]
        }
    ],
    tools=[
        {"type": "web_search"}
    ]
)

BadRequestError: Error code: 400 - {'error': {'message': "Hosted tool 'web_search_preview' is not supported with gpt-4o-2024-11-20.", 'type': 'invalid_request_error', 'param': 'tools', 'code': None}}

In [22]:
import json
print(json.dumps(response_multimodal.__dict__, default=lambda o: o.__dict__, indent=4))

{
    "id": "resp_67bd65392a088191a3b802a61f4fba14",
    "created_at": 1740465465.0,
    "error": null,
    "metadata": {},
    "model": "gpt-4o-2024-08-06",
    "object": "response",
    "output": [
        {
            "id": "msg_67bd653ab9cc81918db973f0c1af9fbb",
            "content": [
                {
                    "annotations": [],
                    "text": "Based on the image of a cat, some relevant keywords could be:\n\n- Cat\n- Feline\n- Pet\n- Animal care\n- Cat behavior\n\nI'll search for recent news related to these keywords.",
                    "type": "output_text",
                    "logprobs": null
                }
            ],
            "role": "assistant",
            "type": "message"
        },
        {
            "id": "ws_67bd653c7a548191af86757fbbca96e1",
            "status": "completed",
            "type": "web_search_call"
        },
        {
            "id": "msg_67bd653f34fc8191989241b2659fd1b5",
            "content": [
           

위 예제에서는 `web_search` 도구를 사용하여 단일 API 호출로 이미지와 관련된 뉴스를 검색할 수 있었습니다. 이는 Chat Completions API를 사용할 경우 여러 번의 왕복 호출이 필요했던 작업을 단순화합니다.

Responses API를 사용하면  
🔥 단일 API 호출로 다음 작업을 처리할 수 있습니다:

✅ 멀티모달 입력을 사용하여 주어진 이미지를 분석합니다.

✅ `web_search` 호스팅 도구를 통해 웹 검색을 수행합니다.

✅ 결과를 요약합니다.

반면, Chat Completions API를 사용하면 여러 단계가 필요하며, 각 단계는 API로의 왕복 호출을 요구합니다:

1️⃣ 이미지를 업로드하고 분석 결과를 얻음 → 1회 요청

2️⃣ 정보를 추출하고 외부 웹 검색 호출 → 수동 단계 + 도구 실행

3️⃣ 도구 결과를 요약을 위해 다시 제출 → 추가 요청

다음 다이어그램에서 시각적으로 비교한 내용을 확인하세요!

![Responses vs Completions](../../images/comparisons.png)

Responses API를 사용해 보고 코드 단순화와 복잡한 멀티모달, 도구 확장 상호작용을 구축하는 데 얼마나 유용한지 확인해 보세요!